In [1]:
import numpy as np
from extensisq import sens_ID
from scipy.integrate import solve_ivp

def fun(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return np.array([-p1*y1 + p2*y2*y3,
                      p1*y1 - p2*y2*y3 - p3*y2**2,
                                         p3*y2**2])

def jac(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return np.array([[-p1, p2*y3, p2*y2],
                     [p1, -p2*y3 - 2*p3*y2, -p2*y2],
                     [0., 2*p3*y2, 0.]])

def dfdp(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return np.array([[-y1, y2*y3, 0.],
                     [y1, -y2*y3, -y2**2],
                     [0., 0., y2**2]])

y0 = np.array([1., 0., 0.])
t_eval = 4 * 10. ** np.arange(-1, 11)
t_span = (0., t_eval[-1])

p = (0.04, 1e4, 3e7)
dy0dp = np.zeros([3, 3])

rtol = 1e-4
atol = np.array([1e-8, 1e-14, 1e-6])

sens, yf, sol = sens_ID(fun, t_span, y0, jac, dfdp, dy0dp, p=p,
                        method='BDF', rtol=rtol, atol=atol,
                        t_eval=t_eval, use_approx_jac=False)

print('nfev:', sol.nfev)
print('njev:', sol.njev)

for t, y in zip(sol.t, sol.y.T):
    print(f'\nT: {t:8.2e}')
    print(f'Solution:     \t{y[0]:>11.4e}\t{y[1]:>11.4e}\t{y[2]:>11.4e}')
    print(f'Sensitivity 1:\t{y[3]:>11.4e}\t{y[4]:>11.4e}\t{y[5]:>11.4e}')
    print(f'Sensitivity 2:\t{y[6]:>11.4e}\t{y[7]:>11.4e}\t{y[8]:>11.4e}')
    print(f'Sensitivity 3:\t{y[9]:>11.4e}\t{y[10]:>11.4e}\t{y[11]:>11.4e}')


nfev: 1306
njev: 29

T: 4.00e-01
Solution:     	 9.8517e-01	 3.3864e-05	 1.4794e-02
Sensitivity 1:	-3.5595e-01	 3.9025e-04	 3.5556e-01
Sensitivity 2:	 9.5416e-08	-2.1310e-10	-9.5203e-08
Sensitivity 3:	-1.5831e-11	-5.2900e-13	 1.6360e-11

T: 4.00e+00
Solution:     	 9.0552e-01	 2.2405e-05	 9.4460e-02
Sensitivity 1:	-1.8761e+00	 1.7921e-04	 1.8760e+00
Sensitivity 2:	 2.9612e-06	-5.8308e-10	-2.9606e-06
Sensitivity 3:	-4.9331e-10	-2.7625e-13	 4.9358e-10

T: 4.00e+01
Solution:     	 7.1582e-01	 9.1852e-06	 2.8417e-01
Sensitivity 1:	-4.2477e+00	 4.5906e-05	 4.2477e+00
Sensitivity 2:	 1.3731e-05	-2.3571e-10	-1.3731e-05
Sensitivity 3:	-2.2884e-09	-1.1380e-13	 2.2885e-09

T: 4.00e+02
Solution:     	 4.5051e-01	 3.2227e-06	 5.4949e-01
Sensitivity 1:	-5.9585e+00	 3.5407e-06	 5.9585e+00
Sensitivity 2:	 2.2739e-05	-2.2592e-11	-2.2739e-05
Sensitivity 3:	-3.7898e-09	-4.9947e-14	 3.7898e-09

T: 4.00e+03
Solution:     	 1.8321e-01	 8.9430e-07	 8.1679e-01
Sensitivity 1:	-4.7500e+00	-5.9926e-06	 4.7500e+

This problem can be solved with LSODA, BDF or Radau as solver. All methods work if use_approx_jac == False and only LSODA seems to work well if use_approx_jac == True.

In [2]:
from extensisq import sens_adjoint_int

def g(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return [y1 + p2*y2*y3]

def dgdy(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return np.array([1., p2*y3, p2*y2])

def dgdp(t, y, *p):
    y1, y2, y3 = y
    p1, p2, p3 = p
    return np.array([0., y2*y3, 0.])

# help(sens_adjoint_int)
t_span = (0., 4e7)
atol_lambda = 1e-5
atol_quad = 1e-6

sens, G, sol_y, sol_bw = sens_adjoint_int(
    fun, t_span, y0, jac, dfdp, dy0dp, p, g, dgdp, dgdy, method='BDF',
    atol=atol, rtol=rtol, atol_quad=atol_quad, atol_lambda=atol_lambda)
lambda0 = sol_bw.y[:3, -1]
print(f'G:\t\t{G:>11.4e}')
print(f'sensitivity:\t{sens[0]:>11.4e}\t{sens[1]:>11.4e}\t{sens[2]:>11.4e}')
print(f'lambda0:\t{lambda0[0]:>11.4e}\t{lambda0[1]:>11.4e}\t{lambda0[2]:>11.4e}')


G:		 1.8194e+04
sensitivity:	-7.8275e+05	 3.1959e+00	-5.3264e-04
lambda0:	 3.4196e+04	 3.4157e+04	 3.4156e+04
